In [1]:
import os
import numpy as np
import torch as tr, torch.nn as nn, torch.utils.data as trdata
import sklearn.metrics as metrics
import time

os.chdir('../')

from model_n_data.dataset import *
import utils
import numpy_augment

print(expe_dir)
print(os.getcwd())
!nvidia-smi

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Python project/abnormal_activities/notebooks/WISDM_active_multitask
/content/drive/My Drive/Python project/abnormal_activities
Sat Apr  3 17:04:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    10W /  70W |      0MiB / 15

In [2]:
def train(model,
          train_set,
          valid_set=None,
          weights_save_name='torch_model',
          only_save_best_of_best=True,
          save_before_early_stop=False,
          curve_save_name=None,
          opt='adam',
          learning_rate=1e-3,
          weight_decay=0.,
          batch_size=32,
          max_epoch=100,
          class_weight=None,
          patience=10,
          multi_gpu=False,
          sample_weight_divide_factor=None,
          ):
    trainloader = trdata.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0)
    if valid_set is not None:
        testloader = trdata.DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=0)

    loss_function = nn.CrossEntropyLoss(reduction='none') if class_weight is None else nn.CrossEntropyLoss(weight=class_weight, reduction='none')

    if opt == 'radam':
        from torch_radam import RAdam
        optimizer = RAdam(params=model.parameters(), lr=learning_rate, weight_decay=weight_decay, )
    elif opt == 'adam':
        optimizer = tr.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    else:
        raise ValueError('opt is adam or radam')

    bestmetric = 0.
    org_patience = patience
    last_epoch_save_name = None

    if multi_gpu:
        num_gpu = tr.cuda.device_count()
        if num_gpu > 1:
            model = nn.DataParallel(model)
            print(f'Let\'s use {num_gpu} GPUs!')
        else:
            print('Only 1 GPU is available :(')

    model = model.cuda()

    if curve_save_name is not None:
        curve_train_loss = []
        curve_valid_loss = []
        curve_train_f1 = []
        curve_valid_f1 = []
        curve_train_acc = []
        curve_valid_acc = []

    for epoch in range(1, max_epoch + 1):
        time_start = time.time()
        model.train()

        epoch_loss = 0.
        n_batch = 0
        ypred_train = []
        ytrue_train = []

        for batch_idx, batch in enumerate(trainloader):
            n_batch += 1

            data, label = batch

            # extract components in label tensor:
            # sample weight
            sample_weight = label[:, 1].float()
            sample_weight = sample_weight/sample_weight.mean()

            # multi-task mask
            multitask_mask = label[:, 2].bool()

            # label
            label = label[:, 0].long()
            
            optimizer.zero_grad()

            trainoutput0, trainoutput1 = model(data, multitask_mask=multitask_mask) # source output, target output

            # print(f"data: source: {data[~multitask_mask][0, 0]}; target: {data[multitask_mask]}")
            # print(f"output shape: source: {trainoutput0.shape}, target: {trainoutput1.shape}")
            # print(sample_weight)
            # print(multitask_mask.sum(), multitask_mask)
            # input(".../")

            loss0 = loss_function(trainoutput0, label[~multitask_mask]) * sample_weight[~multitask_mask]
            loss1 = loss_function(trainoutput1, label[multitask_mask]) * sample_weight[multitask_mask]

            loss = loss0.mean() + loss1.mean()

            loss.backward()

            optimizer.step()

            # calculate trainloss and train ytrue, ypred
            with tr.no_grad():
                epoch_loss += loss
                ypred_train.append(trainoutput1)
                ytrue_train.append(label[multitask_mask])
            if batch_idx == 0:
                print(f"batch sample weight: {np.unique(sample_weight.cpu(), return_counts=True)}")

        # epoch evaluation
        with tr.no_grad():
            epoch_loss /= n_batch
            ytrue_train = tr.cat(ytrue_train, 0).cpu().view(-1).numpy()
            ypred_train = tr.cat(ypred_train, 0).cpu().argmax(1).view(-1).numpy()

            train_f1 = metrics.f1_score(ytrue_train, ypred_train, average='macro')
            train_acc = metrics.accuracy_score(ytrue_train, ypred_train)

            if valid_set is not None:
                model.eval()

                val_epoch_loss = 0.
                ypred_valid = []
                ytrue_valid = []
                n_batch = 0

                for testbatch in testloader:
                    n_batch += 1
                    testdata, testlabel = testbatch
                    testlabel = testlabel.long()
                    
                    _, testoutput = model(testdata, tr.ones(len(testdata)).bool())
                    val_loss = loss_function(testoutput, testlabel).mean()
                    
                    val_epoch_loss += val_loss
                    ypred_valid.append(testoutput)
                    ytrue_valid.append(testlabel)

                val_epoch_loss /= n_batch
                ytrue_valid = tr.cat(ytrue_valid, 0).cpu().view(-1).numpy()
                ypred_valid = tr.cat(ypred_valid, 0).cpu()
                ypred_valid = ypred_valid[:, :6]
                ypred_valid = ypred_valid.argmax(1).view(-1).numpy()

                val_f1 = metrics.f1_score(ytrue_valid, ypred_valid, average='macro')
                val_acc = metrics.accuracy_score(ytrue_valid, ypred_valid)
            else:
                val_epoch_loss = epoch_loss
                val_f1 = train_f1
                val_acc = train_acc

            duration = time.time() - time_start
            print("\nepoch %d, time: %ds\n"
                  "train loss: %.4f\ttrain f1: %.4f\t train acc: %.4f\n"
                  "  val loss: %.4f\t  val f1: %.4f\t   val acc: %.4f"
                  % (epoch, duration, epoch_loss, train_f1, train_acc, val_epoch_loss, val_f1, val_acc))

            if curve_save_name is not None:
                curve_train_loss.append(epoch_loss.item())
                curve_valid_loss.append(val_epoch_loss.item())
                curve_train_f1.append(train_f1)
                curve_valid_f1.append(val_f1)
                curve_train_acc.append(train_acc)
                curve_valid_acc.append(val_acc)

            # model checkpoint
            if val_f1 > bestmetric:
                epoch_name = "%s_%d_%.6f" % (weights_save_name, epoch, val_f1)
                patience = org_patience
                print(
                    f"f1 improved from {bestmetric} to {val_f1}, save to {epoch_name} ---------------------------------")
                tr.save(model.state_dict(), epoch_name)
                bestmetric = val_f1

                if last_epoch_save_name is not None and os.path.exists(last_epoch_save_name) and only_save_best_of_best:
                    os.remove(last_epoch_save_name)
                last_epoch_save_name = epoch_name

            # early stopping
            else:
                patience -= 1
                if patience <= 0:
                    if save_before_early_stop:
                        epoch_name = "%s_%d_%.6f" % (weights_save_name, epoch, val_f1)
                        tr.save(model.state_dict(), epoch_name + '-earlystopping')
                    print('STOPPED-------------------------------------------')
                    break

    if curve_save_name is not None:
        import pandas

        training_curve = np.array([
            curve_train_loss,
            curve_valid_loss,
            curve_train_f1,
            curve_valid_f1,
            curve_train_acc,
            curve_valid_acc,
        ]).T
        cols = ["train_loss",
                "valid_loss",
                "train_f1",
                "valid_f1",
                "train_acc",
                "valid_acc"]
        training_curve = pandas.DataFrame(training_curve, columns=cols)
        training_curve.to_csv(f"{curve_save_name}.txt", index=False, columns=cols)

In [6]:
def load_n_train(augmenter, lr, gamma, sample_weight_divide_factor=-1,
                 weight_file_1=None, weight_file_2=None, weight_file=None, save_name='weight'):

    # LOAD WISDM
    data_folder = f'../Dataset/wisdm'

    train_subjects = np.unique(np.linspace(1, 36, 6, False, dtype=int))
    valid_subjects = np.setdiff1d(np.arange(1, 1+36), train_subjects)
    print(f'num subjects valid/train: {len(valid_subjects)}/{len(train_subjects)}')
    print('valid_subjects', valid_subjects)
    print('train_subjects', train_subjects)

    train_data_n_label = utils.load_dataset_single_file(
        data_file=f'{data_folder}/data_ss.npy',
        p_lb_file=f"{data_folder}/p_lb_ss.npy",
        list_valid_subject_id=valid_subjects,
        is_train_set=True,
        data_channels=[3],
        normalize=False,
        # min_vals=[-8, -150],
        # max_vals=[8, 150]
    )

    valid_data_n_label = utils.load_dataset_single_file(
        data_file=f'{data_folder}/data_ss.npy',
        p_lb_file=f"{data_folder}/p_lb_ss.npy",
        list_valid_subject_id=valid_subjects,
        is_train_set=False,
        data_channels=[3],
        normalize=False,
        # min_vals=[-8, -150],
        # max_vals=[8, 150]
    )

    traindata = train_data_n_label[0]
    trainlabel = train_data_n_label[1]
    validdata = valid_data_n_label[0]
    validlabel = valid_data_n_label[1]

    # SET WEIGHT PLACEHOLDER FOR INSTANCES FROM TARGET DOMAIN
    pholder = np.ones(shape=[len(trainlabel), 3], dtype=float)
    pholder[:, 0] = trainlabel
    trainlabel = np.copy(pholder)

    # LOAD MOBIACT V2
    data_folder = '../Dataset/mobiact_v2'
    
    pick_data_n_label = utils.load_dataset_single_file(
        data_file=f'{data_folder}/data_new.npy',
        p_lb_file=f"{data_folder}/p_lb_ss_new.npy",
        list_valid_subject_id=[],
        is_train_set=True,
        data_channels=[3, 3],
        normalize=False,
        lb_col_in_p_lb=1,
        # min_vals=[-8, -150],
        # max_vals=[8, 150]
    )
    pick_index = np.load('notebooks/MotionSense_active/mobiactv2_r2_pick_index_one-tenth.npy')
    
    pickdata = pick_data_n_label[0]
    picklabel = pick_data_n_label[-1]

    # SET WEIGHT FOR INSTANCES FROM SOURCE DOMAIN
    pholder = np.ones(shape=[len(picklabel), 3], dtype=float)
    pholder[:, 0] = picklabel
    picklabel = np.copy(pholder)

    # SET WEIGTH FOR SOURCE AND TARGET LABELS
    total_len = len(trainlabel) + len(pick_index)
    source_weight = (len(pick_index) / total_len) ** -1
    target_weight = (len(trainlabel) / total_len) ** -1
    
    target_weight *= gamma

    print(f"source size: {len(pick_index)}, target size: {len(trainlabel)}, total size: {total_len}")
    print(f"source weight, target weight: {source_weight}, {target_weight}")

    picklabel[:, 1] = source_weight
    trainlabel[:, 1] = target_weight

    # SET MULTI-TASK MASK
    picklabel[:, 2] = 0 # source mask
    trainlabel[:, 2] = 1 # target mask

    # MERGE DATA OF SOURCE AND TARGET DOMAIN
    traindata = np.concatenate([traindata, pickdata[pick_index]])

    n_class_source = len(np.unique(picklabel[:, 0]))
    n_class_target = len(np.unique(trainlabel[:, 0]))
    print(f"num class source: {n_class_source}; target: {n_class_target}")
    
    trainlabel = np.concatenate([trainlabel, picklabel[pick_index]])

    train_set = DatasetWindows(traindata, trainlabel, augment_rate=0.5, augmenter=augmenter)
    valid_set = DatasetWindows(validdata, validlabel, augment_rate=0.)

    from model_n_data.model_tcn_classifier import TCN
    model = TCN(n_classes=[n_class_source, n_class_target],  # type:int
                how_flatten="spatial attention gap",
                how_between_tcn_blocks="none",
                n_tcn_channels=(64,) * 6 + (128,) * 2,  # type: list, tuple
                tcn_kernel_size=2,  # type:int
                dilation_base=2,  # type:int
                tcn_droprate=0.2,  # type: float
                use_spatial_dropout=False,
                n_fc_layers=1,
                fc_droprate=0.8,  # type: float
                use_init_batchnorm=True,
                use_last_fc=True,
                )
    model.cuda()
    if weight_file is not None:
        print('load w', weight_file)
        model.load_state_dict(tr.load(weight_file))

    print('------------------------------')
    # print(model)
    print('------------------------------')
    print(f"params: {utils.torch_num_params(model)}")
    print(f"train data: {traindata.shape}")
    print(f"train label: {trainlabel.shape}")
    print(f"val data: {validdata.shape}")
    print(f"val label: {validlabel.shape}")

    print(traindata.max())
    print(validdata.max())
    print(traindata.min())
    print(validdata.min())
    print()

    train(model,
          train_set=train_set,
          valid_set=valid_set,
          weights_save_name=f'{expe_dir}/{save_name}',
          only_save_best_of_best=True,
          save_before_early_stop=False,
          curve_save_name=None,
          opt='adam',
          learning_rate=lr,
          weight_decay=0.,
          batch_size=32,
          max_epoch=100,
          class_weight=None,
          patience=10,
          multi_gpu=False,
          sample_weight_divide_factor=sample_weight_divide_factor
    )

In [4]:
# augmenter = augment.Augmenter(
#         input_shape=[300,3],
#         augmentation_apply_rate=1,

#         max_rotate_x=20, max_rotate_y=20, max_rotate_z=20,
#     )
augmenter = numpy_augment.NumpyAugmenter(
    input_shape=[300, 3],
    max_num_transformations=3,
    shuffle_transformations=False,

    rotate_x_range=[0., 20.],
    rotate_y_range=[0., 20.],
    rotate_z_range=[0., 20.],
    # magnitude_warp_sigma_range=[0., 0.1],
    # nums_magnitude_warp_knots=[3,4],
    # time_warp_sigma_range=[0., 0.2],
    # nums_time_warp_knots=[3,4]
)

In [7]:
load_n_train(
    save_name='r1_auto_weight_gamma1',
    lr=1e-3,
    gamma=1.,
              augmenter=augmenter,
              )

num subjects valid/train: 30/6
valid_subjects [ 2  3  4  5  7  8  9 10 11 13 14 15 16 17 19 20 21 22 23 25 26 27 28 29
 31 32 33 34 35 36]
train_subjects [ 1  6 12 18 24 30]
source size: 11164, target size: 7553, total size: 18717
source weight, target weight: 1.6765496237907558, 2.4780881768833574
num class source: 16; target: 6
------------------------------
------------------------------
params: 223047.0
train data: (18717, 300, 3)
train label: (18717, 3)
val data: (41682, 300, 3)
val label: (41682,)
19.61
20.04
-19.61
-19.68087

batch sample weight: (array([0.81692415, 1.2074859 ], dtype=float32), array([17, 15]))

epoch 1, time: 24s
train loss: 3.8452	train f1: 0.6808	 train acc: 0.7160
  val loss: 1.3656	  val f1: 0.6320	   val acc: 0.6750
f1 improved from 0.0 to 0.6320004718073585, save to /content/drive/My Drive/Python project/abnormal_activities/notebooks/WISDM_active_multitask/r1_auto_weight_gamma1_1_0.632000 ---------------------------------
batch sample weight: (array([0.82